In [1]:
# Import packages
import numpy as np
import pandas as pd
import sys
from datetime import datetime, timedelta
import time
from apscheduler.schedulers.blocking import BlockingScheduler
sys.path.append('../..')
import pickle

import h5py
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, MaxPooling2D, Conv2D
from keras.callbacks import ModelCheckpoint, Callback
import matplotlib.pyplot as plt
import missingno as msno

from Classifier.data_processing import processor

Using TensorFlow backend.


## Define and Initialise Prediction Model

In [2]:
# Define prediction model
class LSTM_net:
    """
    RNN using LSTM
    """
    def __init__(self, input_size, learning_rate):
        self.input_size = input_size
        self.learning_rate = learning_rate
        self.build_model()

    def build_model(self):
        self.model = Sequential()
        self.model.add(LSTM(256, return_sequences=True,
                       input_shape=self.input_size))  
        #self.model.add(Dropout(0.2))
        self.model.add(LSTM(256))  
        #self.model.add(Dropout(0.2))
        self.model.add(Dense(1, activation='linear'))

        # Define optimiser and compile
        optimizer = optimizers.Adam(self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

In [3]:
# Initialise model
sequence_length = 4
input_size = 35
learning_rate = 0.00001 # Only needed to define optimiser. Not used in prediction
input_size = (sequence_length, input_size)
LSTM_network = LSTM_net(input_size, learning_rate)

# Load the model weights with the best validation loss.
LSTM_network.model.load_weights('saved_models/LSTM_weights.hdf5')

In [4]:
# Load normalisation data
with open('pickles/normalisation.pickle', 'rb') as f:
    x_mean, x_std, y_mean, y_std = pickle.load(f)

## Define data download and processing function

In [5]:
interval = 1440 # 1440 minutes = 1 day

def data_feed(interval, sequence_length, x_mean, x_std):
    """
    Function to download most recent data, scrub and normalise it, and convert to sequential values for LSTM
    :interval: Time period in minutes between datapoints. Needs to be same as for original training data
    :sequence_length: Number of intervals of "memory" for LSTM network. Data will be fed in time windows of length=sequence_length
    :x_mean: training/validation data mean values for normalisation
    :x_std: training/validation data standard deviation values for normalisation
    Returns tuple of np.array data for last time window. Shape of (1, sequence_length, input_size), and current price for logging purposes
    """
    
    # Download data for the most recent period
    end_time = datetime.now().replace(microsecond=0,second=0,minute=0)
    start_time = end_time - timedelta(minutes=interval * (sequence_length + 1)) # Adding some historical data in case interpolation needed
    data = processor.historical_download(start_time, end_time, interval)
    
    # Convert to float and interpolate any missing values
    data = data.astype('float64')
    data = data.interpolate()
    target = "Btcusd_kraken_close" # Only used for training. Must be the same as for training
    current_price = data[target] # For logging purposes
    
    # Convert to growth rates and np.arrays
    data = data.pct_change()
    x = np.array(data[1:]) # First value removed. Will always be NaN because growth rates
        
    # Normalise data
    x = (x - x_mean) / x_std
    
    # Reshape data from (num_samples, features) to (num_samples, sequence_length, features)
    seq_x = []
    for ii in range(len(x) - sequence_length + 1):
        seq_x.append(x[ii : ii + sequence_length])
    
    seq_x = np.array(seq_x)
    
    input_data = np.reshape(seq_x[-1], (-1, sequence_length, seq_x.shape[2]))

    return input_data, current_price[-1]

## Set up trade scheduler

In [6]:
live_trading = False
# Paper trading balances
if not live_trading:
    cash = 1000
    btc = 0

In [ ]:
sched = BlockingScheduler()

@sched.scheduled_job('cron', day_of_week='mon-sun', hour=10, minute=13, timezone='UTC')
def trade():
    # Run prediction script
    Date = datetime.now()
    input_data, current_price = data_feed(interval, sequence_length, x_mean, x_std)
    raw_prediction = LSTM_network.model.predict(input_data)
    expected_growth = raw_prediction.item() * y_std + y_mean
    predicted_price = current_price * (1 + expected_growth)

    # Simple trading algorithm.
    if expected_growth > 0:
        action = "BUY"
        position += cash / test_actuals_for_seq[ii] * 0.997 # 0.997 to account for fees
        cash = 0
    if expected_growth < 0:
        action = "SELL"
        cash += position * test_actuals_for_seq[ii] * 0.997
        position = 0

    if live_trading:
        # call Kraken API to make trades here
        pass

    # Reporting and logging
    print("{}: {}. Price expected to change from {} to {}. Portfolio value of {}".format(Date, action, current_price, predicted_price))    
    log = pd.DataFrame([Date, action, current_price, predicted_price, cash, btc, (cash + btc * current_price)], columns=["Date", "Action", "Current Price", "Predicted_Price", "Cash_Position", "BTC_Position", "Portfolio_Value"])
    log.to_csv('trade_log.csv', encoding='utf-8', index=True)

sched.start()

time period from 2018-03-14 02:36:24.003970 to 1521005107.9140759
time period from 2018-03-14 12:25:07.914076 to 1521035506.7856727
time period from 2018-03-14 20:51:46.785673 to 1521063034.0363095
time period from 2018-03-15 04:30:34.036309 to 1521092498.1881964
time period from 2018-03-15 12:41:38.188196 to 1521122946.0273845
time period from 2018-03-15 21:09:06.027385 to 1521150701.0972052
time period from 2018-03-16 04:51:41.097205 to 1521181300.0006766
time period from 2018-03-16 13:21:40.000677 to 1521209692.0892293
time period from 2018-03-16 21:14:52.089229 to 1521241702.7070386
time period from 2018-03-17 06:08:22.707039 to 1521272636.5180721
time period from 2018-03-17 14:43:56.518072 to 1521302217.4866335
time period from 2018-03-17 22:56:57.486634 to 1521331815.3039234
time period from 2018-03-18 07:10:15.303923 to 1521365452.2784066
time period from 2018-03-18 16:30:52.278407 to 1521390652.0
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time pe

/Users/alastairong/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2018-03-14 00:49:45.980301 to 1520990771.7059102
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-03-14 08:26:11.705910 to 1521019111.7258077
time period from 2018-03-14 16:18:31.725808 to 1521045473.2742498
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2018-03-14 23:37:53.274250 to 1521073095.834538
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-03-15 07:18:15.834538 to 1521100893.640569
time period from 2018-03-15 15:01:33.640569 to 1521126567.1723964
call rate limiter exceeded (counter=21, limit=20) 
 sleeping for 5 seconds
time period from 2018-03-15 22:09:27.172396 to 1521154320.8622515
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time period from 2018-03-16 05:52:00.862252 to 1521182984.4422984
time period from 2018-03

Job "trade (trigger: cron[day_of_week='mon-sun', hour='10', minute='13'], next run at: 2018-03-19 10:13:00 UTC)" raised an exception
Traceback (most recent call last):
  File "/Users/alastairong/anaconda3/lib/python3.6/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-7-8cc05f5b8c02>", line 8, in trade
    raw_prediction = LSTM_network.model.predict(input_data)
  File "/Users/alastairong/anaconda3/lib/python3.6/site-packages/keras/models.py", line 1025, in predict
    steps=steps)
  File "/Users/alastairong/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1832, in predict
    self._make_predict_function()
  File "/Users/alastairong/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1029, in _make_predict_function
    **kwargs)
  File "/Users/alastairong/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2502, in function
    return